In [36]:
from langsmith import Client
import os
client = Client()
project_name=""
def _set_if_undefined(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"Please provide your {var}")
_set_if_undefined("OPENAI_API_KEY")
_set_if_undefined("LANGCHAIN_API_KEY")
_set_if_undefined("LANGCHAIN_API_YI_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.environ["LANGCHAIN_API_YI_KEY"]

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "xmode-vqa-gpt_4o-english-100-with-intent"
project_name=os.environ["LANGCHAIN_PROJECT"]
project_runs = list(client.list_runs(project_name=project_name, is_root=True))

In [37]:
# Filter out only successful runs
success_project_runs = [run for run in project_runs if run.status == "success"]

In [49]:
success_project_runs[0].__dict__

{'id': UUID('a0f3bcfa-80f8-454a-bce5-ca1b2070df4e'),
 'name': 'LangGraph',
 'start_time': datetime.datetime(2024, 12, 8, 11, 49, 47, 462715),
 'run_type': 'chain',
 'end_time': datetime.datetime(2024, 12, 8, 11, 50, 1, 905567),
 'extra': {'runtime': {'sdk': 'langsmith-py',
   'sdk_version': '0.1.96',
   'library': 'langsmith',
   'platform': 'Linux-5.15.0-40-generic-x86_64-with-glibc2.35',
   'runtime': 'python',
   'py_implementation': 'CPython',
   'runtime_version': '3.10.8',
   'langchain_version': '0.2.12',
   'langchain_core_version': '0.2.27',
   'thread_count': 10.0,
   'mem': {'rss': 224436224.0},
   'cpu': {'time': {'sys': 6.83, 'user': 50.09},
    'ctx_switches': {'voluntary': 326.0, 'involuntary': 18.0},
    'percent': 0.0}},
  'metadata': {'revision_id': '8f29e69-dirty'}},
 'error': None,
 'serialized': None,
 'events': [{'name': 'start', 'time': '2024-12-08T11:49:47.462715+00:00'},
  {'name': 'end', 'time': '2024-12-08T11:50:01.905567+00:00'}],
 'inputs': {'input': [{'con

In [33]:
assert len(success_project_runs)==100

In [31]:
def extract_run(run):
    question = run.inputs["input"][0]["content"][0]["question"]
    inputs = run.inputs["input"]
    outputs = run.outputs["output"]
    prompt_tokens = run.prompt_tokens
    completion_tokens = run.completion_tokens
    total_tokens = run.total_tokens
    total_cost = float(run.total_cost)
    prompt_cost = float(run.prompt_cost)
    completion_cost = float(run.completion_cost)
    return {
        "question": question,
        "inputs": inputs,
        "outputs": outputs,
        "prompt_tokens": prompt_tokens,
        "completion_tokens": completion_tokens,
        "total_tokens": total_tokens,
        "total_cost": total_cost,
        "prompt_cost": prompt_cost,
        "completion_cost": completion_cost,
    }

res = []
for run in success_project_runs:
    res.append(extract_run(run))


In [ ]:
import json
with open("experiments/xmode/en/metadata_xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json", "w") as f:
    json.dump(res[::-1], f, indent=2) # reverse the order to match the order of the questions in the dataset

In [38]:
with open("experiments/xmode/en/metadata_xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json", "r") as f:
    res = json.load(f)


In [41]:
data_path = "/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent.json"
with open(data_path, "r") as f:
    data = json.load(f)

for d,r in zip(data,res):
    assert d["question"]==r["question"], print(d["question"],r["question"])
    d["prompt_tokens"] = r["prompt_tokens"]
    d["completion_tokens"] = r["completion_tokens"]
    d["total_tokens"] = r["total_tokens"]
    d["prompt_cost"] = r["prompt_cost"]
    d["completion_cost"] = r["completion_cost"]
    d["total_cost"] = r["total_cost"]

merged_data_path = "/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.json"
with open(merged_data_path, "w") as f:
    json.dump(data, f, indent=2)

import pandas as pd
df = pd.DataFrame(data)
df.to_csv("/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.csv", index=False)


In [42]:
df.to_excel("/home/ubuntu/workspace/XMODE-LLMCompiler/experiments/xmode/en/xmode-vqa-m3ae-star-100-en-gpt_4o-with-intent-with-meta.xlsx", index=False)

"given the last study of patient 10284038 in 2105, is the cardiac silhouette's width larger than half of the total thorax width?"